<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [98]:
#Загружаем необходимые модули для дальнейшей работы.
import pandas as pd
import psycopg2
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select COUNT(distinct id)
from VACANCIES
'''

In [5]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

/tmp/ipykernel_4497/4045908245.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_1 = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
select COUNT(distinct id)
from EMPLOYERS
'''

In [7]:
# результат запроса
df_3_2  = pd.read_sql_query(query_3_2, connection)
df_3_2 

/tmp/ipykernel_4497/2391532654.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_2  = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
select COUNT(distinct id) 
from AREAS
'''

In [9]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

/tmp/ipykernel_4497/1395976625.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_3 = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
select COUNT(distinct id)
from INDUSTRIES
'''

In [11]:
# результат запроса
df_3_4= pd.read_sql_query(query_3_4, connection)
df_3_4

/tmp/ipykernel_4497/194207669.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_4= pd.read_sql_query(query_3_4, connection)


,count
0,294


5. Сделаем запрос, где будет видно распределение графиков работы по всем анкетам в HH.ru

In [114]:

query_3_5 = f'''
select v.schedule, count(*) as "Count"
from VACANCIES as "v"
left join AREAS as "a" on v.area_id = a.id
left join EMPLOYERS as "e" on v.employer_id = e.id
--left join EMPLOYERS_INDUSTRIES as "e_i" on v.employer_id = e_i.employer_id  
group by v.schedule
order by 2 desc
'''
df_3_5 = pd.read_sql_query(query_3_5, connection)

fig = px.bar(
    data_frame=df_3_5,
    x = 'schedule',
    y = 'Count',
    height=400,
    width=800,
    title='Распределение рабочих графиков всех анкет на HH.ru',
    text='Count'
);
fig.show()

/tmp/ipykernel_4497/2659310307.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



<img src=3_5.png>

# выводы по предварительному анализу данных 
#### Что мы видим из запросов выше
* У нас есть 49197 вакансий на портале которые были размещены 23501 работодателем в 1362 регионах и при этом 294 сферы деятельности.
* Учитывая, что на 1 января 2024 года в России насчитывается 1120 городов, можно предположить, что вакансии не только России размещены на данном портале.
* В среднем на одного работодателя приходится две вакансии. Я уверен, что у некоторых работодателей их более.
* Так же видно, что количество работодателей, кто готов предоставить удаленную работу заметно увеличился после пандемии. 
* Но по прежнему полный рабочий день в офисе на первом месте.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''
select a.name as "area",count(distinct v.id) as "cnt"
from VACANCIES as  "v"
join AREAS as "a" on v.area_id = a.id
group by 1
order by 2 desc

'''

In [14]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

/tmp/ipykernel_4497/620650199.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Шахтерск (Сахалинская область),1
765,Пестово,1
766,Пересвет,1
767,Первомайское (Ленинградская область),1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''
select count(distinct id)
from VACANCIES 
where (salary_from is not null) or (salary_to is not null)

'''

In [16]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2

/tmp/ipykernel_4497/3746829199.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''
select round(avg(salary_from)) as "AVG_MIN",round(avg(salary_to)) as "AVG_MAX"
from VACANCIES 
where (salary_from is not null) or (salary_to is not null)

'''

In [18]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

/tmp/ipykernel_4497/3595259731.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,AVG_MIN,AVG_MAX
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f'''
select schedule,employment,count(*) 
from VACANCIES
group by schedule,employment
order by count(*) desc
offset 1
limit 1 
'''

In [20]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

/tmp/ipykernel_4497/2028972612.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,count
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''
select experience,count(distinct id) 
from VACANCIES
group by experience
order by count(distinct id) 

'''

In [22]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

/tmp/ipykernel_4497/210463167.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Рассмотрим средние зарплаты по регионам и максимальные значения.

6. Топ 10. Среднее значение зарплат 

In [23]:
query_4_6= f'''
select n.name, round(avg(case 
    when n.salary_from is null and n.salary_to is not null  then n.salary_to
    when n.salary_from is not null  and n.salary_to is null then n.salary_from
    when n.salary_from is not null and n.salary_to is not null then (n.salary_from + n.salary_to)/2
    end)) as "resoult"                   
from
(select a.name,coalesce(salary_from,0) as "salary_from",coalesce(salary_to,0) as "salary_to"
from VACANCIES as "v"
left join AREAS as "a" ON v.area_id = a.id 
where salary_from <> 0 or salary_to <> 0) as "n"
group by n.name
order by 2 desc
limit 10
'''
df_4_6 = pd.read_sql_query(query_4_6, connection)
df_4_6

/tmp/ipykernel_4497/125980600.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,resoult
0,Германия,452751.0
1,Черногория,259365.0
2,Дзержинск (Беларусь),220349.0
3,Великобритания,208962.0
4,Болгария,203841.0
5,Заполярный (Ямало-Ненецкий АО),200000.0
6,Кутаиси,195475.0
7,Турция,192747.0
8,Таджикистан,173302.0
9,Нидерланды,168750.0


7. Топ 10. Регионы, где максимальные уровни зарплат самые высокие.

In [24]:
query_4_7 = f'''
select a.name,max(case 
    when v.salary_from is null and v.salary_to is  null  then 0
    when v.salary_from is null and v.salary_to is not null  then v.salary_to
    when v.salary_from is not null  and v.salary_to is null then v.salary_from
    when v.salary_from is not null and v.salary_to is not null then (v.salary_from + v.salary_to)/2
    end) as "resoult"
from VACANCIES as "v"
left join AREAS as "a" ON v.area_id=a.id
group by a.name
order by 2 desc
limit 10
'''
df_4_7 = pd.read_sql_query(query_4_7, connection)
df_4_7

/tmp/ipykernel_4497/683305405.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,resoult
0,Санкт-Петербург,1000000
1,Сочи,750000
2,Москва,750000
3,Алматы,717000
4,Пермь,600000
5,Уфа,600000
6,Минск,526036
7,Иваново (Ивановская область),500000
8,Казань,500000
9,Чебоксары,500000


8. На основании запроса выше, выведем компании, которые готовы платить зарплату в 1 млн. рублей.

In [25]:
query_4_8 = f'''
select v.name,v.key_skills,v.salary_from,v.salary_to,a.name as "city_name"  
from VACANCIES as "v"
left join AREAS as "a" on v.area_id = a.id
where salary_from >=1000000 or salary_to >= 1000000
'''
df_4_8 = pd.read_sql_query(query_4_8, connection)
df_4_8

/tmp/ipykernel_4497/4164873761.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,key_skills,salary_from,salary_to,city_name
0,Технический директор,None,1000000,NaN,Санкт-Петербург
1,QA Automation Engineer,Тестирование\tHTML\tJavaScript\tGit\tТестирова...,400000,1000000.0,Алматы
2,СТО/Директор по разработке,Laravel\tУправление проектами\tPHP\tVue.js\tPo...,500000,1000000.0,Москва
3,Брокер по продаже апартаментов,Ведение переговоров\tТелефонные переговоры\tРа...,500000,1000000.0,Сочи


9. Запрос, где видно средний уровень зарплат в регионах-миллиониках

In [62]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
links = page.find_all('div', class_='ts-ПозКарта-метка')
lst_city= []
for link in links:
    lst_city.append(link.text)  
lst_city=tuple(lst_city)
lst_city

query_4_9= f'''
select *
from
(select a.name,case 
    when v.salary_from is null and v.salary_to is not null then v.salary_to
    when v.salary_from is not null and v.salary_to is null then v.salary_from
    when v.salary_from is not null and v.salary_to is not null then (v.salary_from + v.salary_to)/2
    end as "resoult"
from VACANCIES as "v"
left join AREAS as "a" ON v.area_id=a.id) as "n"
where (n.name in {lst_city} ) 
and n.resoult is not null    


'''
df_4_9 = pd.read_sql_query(query_4_9, connection)
fig =px.box(
    data_frame=df_4_9,
    y='resoult',
    x='name',
    height=600,
    width=900,
    title='уровень зарплат в регионах-миллионниках',
    color='name',
    orientation='v');
fig.show()


/tmp/ipykernel_4497/2433234154.py:28: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



***

<img src=4_90.png>

# выводы по детальному анализу вакансий 
#### Что мы видим из запросов выше
* Пятерка городов, где требуется больше всего работников:Москва, Санкт-Петербург,Минск, Новосибирск и Алматы. Как я ранее говорил, регионы не только России    представлены на HH.ru(Если убрать Минск и Алматы, то следующими идут Екатеринбург и Нижний Новгород). 
* Так же видим, что у половины вакансий не указан хотя бы один из порогов зарплаты, что усложняет анализ. Тут может быть несколько причин, почему не указывается зп.
* Порог очень низкий, либо есть вилка и хотят услышать вариант соискателя, тем самым есть возможность сэкономить, либо сомнительная вакансия.
* Мы видим, средний уровень зарплаты на основе наших вакансий в HH.ru довольно приличный : около 90 тыс. чуть меньше 1000$. Конечно хотелось бы больше.
* Так же видим, что из всех вакансий самые высокий средний уровень зарплаты в Германии. А вот из Российских регионов Заполярный (Ямало-Ненецкий АО) может  похвастаться-200 тыс.
* Но если брать во внимание, что в иностранных регионах выставлено по 2-3 вакансии, то картина не реалистичная. 
* Взяв максимальные значения зарплат сразу видно, что уровень зп. в миллиониках выше, а максимальные значения гораздо выше средних.
* Из запроса выше, где указаны вакансии с зарплатами 1 млн. мы видим, что даже никаких нет требований к кандидату с такой зарплатой. Возможно выброс. И следом на графике отчетливо видно, что значение в 1млн. рублей явный выброс, т.к. на графике он находится одиноко далеко от всех значений, если даже сравнивать с Москвой, где единичные значения идут очень близко к друг другу.
* Есть и программисты, но и брокером в Сочи можно хорошие деньги зарабатывать. 
* Есть еще один город, который доказывает, что в регионах высоких уровень зп. ИВАНОВО.- максимальное значение 500 тыс. 
* Так же нельзя не отметить тот факт, что все больше работодателей готовы предоставить удаленный график работы, т.к. офисную работу можно заменить на работу из дома. Данный тренд появился после пандемии и продолжает набирать популярность. Т.к. в данном случае работодатель может съэкономить на офисных помещениях.
* Интересен тот факт, что почти 15% из всего количества вакансий опыт работы не важен или его нет. Есть куда идти студентам и новичкам. 
* А вот опыт работы от 1 года до 3х самый популярный, т.к. этого периода досточно, как видит работодатель, что бы понять все нюансы профессии. Тут радует тот факт, что учусь и практикуюсь я 2 года, а значит при упорной работе, есть шанс понять все нюасы профессии.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [63]:
# текст запроса
query_5_1 = f'''
(select e.name,count(distinct v.id)
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id
group by e.name
order by count(distinct v.id) desc
limit 1) 
union 
(select e.name,count(distinct v.id)
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id
group by e.name
order by count(distinct v.id) desc
offset 4
limit 1)
order by 2 desc
'''

In [64]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

/tmp/ipykernel_4497/1147638897.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [29]:
# текст запроса
query_5_2 = f'''
select a.name as "city_name", count(e.id ) as "count_employers" , count(v.id) as "count_vacancies"
from AREAS as "a"
left join EMPLOYERS as "e" on a.id=e.area
left join VACANCIES as "v" on a.id = v.area_id
group by a.name
order by 3 , 2 desc
limit 1


'''

In [30]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

/tmp/ipykernel_4497/2306256317.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,city_name,count_employers,count_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [31]:
# текст запроса
query_5_3 = f'''
select  e.name as "employers" ,count(distinct v.area_id) as "count_area"
from VACANCIES as "v" 
join EMPLOYERS as "e" on v.employer_id = e.id
group by e.name
order by 2 desc



'''

In [32]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

/tmp/ipykernel_4497/133400711.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers,count_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [33]:
# текст запроса
query_5_4 = f'''
select count(*)
from 
(select e.name, count(ei.industry_id) as "count_industry"
from EMPLOYERS as "e"
left join EMPLOYERS_INDUSTRIES as "ei" on e.id = ei.employer_id 
group by e.id) as "t"
where t.count_industry = 0


'''

In [34]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

/tmp/ipykernel_4497/1045119256.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [35]:
# текст запроса
query_5_5 = f'''
select c.name_company 
from
(select e.name as "name_company",count(ei.industry_id) as "count_industry"
from 
EMPLOYERS as "e"
join EMPLOYERS_INDUSTRIES as "ei" on e.id = ei.employer_id 
group by e.id) as "c"
where c.count_industry = 4 
order by c.name_company 
offset 2 
limit 1


'''

In [36]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

/tmp/ipykernel_4497/4025176131.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name_company
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [37]:
# текст запроса
query_5_6 = f'''
select  count(distinct ei.employer_id )
from EMPLOYERS_INDUSTRIES as "ei"
left join INDUSTRIES as "i" on ei.industry_id = i.id 
where i.name = 'Разработка программного обеспечения'


'''

In [38]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6


/tmp/ipykernel_4497/1333455304.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим заданием, посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [39]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
links = page.find_all('div', class_='ts-ПозКарта-метка')
lst_city= []
for link in links:
    lst_city.append(link.text)  
lst_city=tuple(lst_city)
lst_city


('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Красноярск',
 'Екатеринбург',
 'Нижний Новгород',
 'Самара',
 'Казань',
 'Пермь',
 'Омск',
 'Челябинск',
 'Ростов-на-Дону',
 'Волгоград',
 'Уфа',
 'Воронеж',
 'Краснодар')

In [65]:
# текст запроса
query_5_7 = f'''
(select a.name , count(v.id) as "cnt"
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id 
join AREAS as "a" on v.area_id = a.id
where (e.name = 'Яндекс') and a.name in {lst_city } 
group by a.name
order by 2)
union all
select 'total',sum(d.cnt)
from 
(select a.name , count(v.id) as "cnt"
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id 
join AREAS as "a" on v.area_id = a.id
where (e.name = 'Яндекс') and a.name in {lst_city}
group by a.name
order by 2) as "d"


'''

In [66]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

/tmp/ipykernel_4497/2595257991.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


8. Сделаем запрос на компании в регионах-миллионниках, у которых вакансий больше 15 на портале и выведем средние их зарплаты. В порядке убывания уровня зарплат.

In [103]:
# текст запроса
query_5_8 = f'''
select e.name,
    avg((case
    when v.salary_from is  null and v.salary_to is not null then v.salary_to
    when v.salary_from is not null and v.salary_to is null then v.salary_from
    when v.salary_from is not null and v.salary_to is not null then (v.salary_from + v.salary_to)/2
    end)) as "average_salary" 
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id 
join AREAS as "a" on v.area_id = a.id
where (v.salary_from is not null or v.salary_to is not null) and a.name in {lst_city }  
group by e.name

order by 2 desc


'''

In [104]:
df_5_8 = pd.read_sql_query(query_5_8, connection)
df_5_8

/tmp/ipykernel_4497/38298331.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,average_salary
0,Alpha Personnel,1000000.0
1,ЦТП,650000.0
2,RuWork,555261.0
3,Пикабу,550000.0
4,BeRecruited,550000.0
...,...,...
5279,Риот,10000.0
5280,Гуляк Андрей,9999.5
5281,GoDoc,6000.0
5282,A5000 Event Solutions,4500.0


* Много компаний, у которых очень высокие зарплаты. Как раз и видны те самые выбросы, которые мы видели ранее. В этом запросе видим названия компания с такими зарплатами.

9. Сделаем запрос, где будет указана средняя зарплата в компаниях, в регионах-миллионниках, у кого в этих регионах размещено не менее 15 вакансий.

In [79]:
query_5_9 = f'''
select e.name,
    count(*) as "number_of_vacancies",
    round(avg(case
    when v.salary_from is  null and v.salary_to is not null then v.salary_to
    when v.salary_from is not null and v.salary_to is null then v.salary_from
    when v.salary_from is not null and v.salary_to is not null then (v.salary_from + v.salary_to)/2
    end)) as "average_salary" 
from EMPLOYERS as "e"
join VACANCIES as "v" on e.id=v.employer_id 
join AREAS as "a" on v.area_id = a.id
where (v.salary_from is not null or v.salary_to is not null) and a.name in ('Москва','Санкт-Петербург','Новосибирск','Красноярск','Екатеринбург','Нижний Новгород','Самара',
 'Казань','Пермь','Омск','Челябинск','Ростов-на-Дону','Волгоград','Уфа','Воронеж','Краснодар') 
group by e.name
having count(*) >=20
order by 3 desc,2 desc

'''

In [102]:
df_5_9 = pd.read_sql_query(query_5_9, connection)
fig = px.bar(
    data_frame=df_5_9,
    x='name',
    y='average_salary',
    title='Средняя зарплата компаний в регионах-миллионниках, у которых на портале размещено не менее 15 объявлений'
);
fig.show()


/tmp/ipykernel_4497/504928162.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



***

<img src=5_9.png>

# выводы по анализу работодателей
#### Что мы видим из запросов выше
* Видим, что на первом месте с сильным отрывом расположился Яндекс	(1933 вакансии),Ростелеком	491,Тинькофф	444,СБЕР	428,Газпром нефть	331
* Есть регионы, в которых отсутсвуют вакансии и судя по запросу такие регионы как в целом Россия, Казахстан, Московская область, Краснодарский край и Беларусь.
* Компания Яндекс разместила поиск соискателей в 181 регионе, что является показателем размера компании. И в регионах-миллионниках замещено 485 вакансий на разные должности
* Из отчета видно, что 8419 компаний решили не указывать свою сферу деятельности. Около 16% из всех вакансий не указана сфера деятельности компании, это не внушает доверие к компании, потому что работа без ОКВЭД запрещена законом и плюс очень много мошенников на рынке. Это огромный минус для компании.
* 4991 компания имеет более 2х сфер деятельности.
* У кого есть желание заняться разработкой программного обеспечения нашлось для него 3553 вакансий.
* Но даже при том, что компания Яндекс является крупным игроком на рынке, в регионах-миллиониках среди компаний, у которых размещено более 15 объявлений, она на последнем месте по среднему уровню зарплат. Много компаний, кто предлагает достойную зарплату, особенно это касается сферы IT.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [42]:
# текст запроса
query_6_1 = f'''
select count(id)
from VACANCIES
where (lower(name) Like '%data%') or (lower(name) Like '%данн%')


'''

In [43]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

/tmp/ipykernel_4497/1281472032.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [44]:
# текст запроса
query_6_2 = f'''
select count(name)
from 
(select *
from VACANCIES
where name ilike '%data%scientist%' 
or name ilike '%data%science%'
or name ilike '%исследователь%данных%'
or (name like '%ML%' and  name not ilike '%HTML%')
or name ilike '%machine%learning%'
or name ilike '%машинн%обучен%'
) as "t"
where (t.name ilike '%junior%' or t.experience ilike '%нет%опыта%' or t.employment ilike '%стажировка%') 


'''

In [45]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

/tmp/ipykernel_4497/118162426.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [46]:
# текст запроса
query_6_3 = f'''
select count(*) 
from VACANCIES
where
(name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
or (name ilike '%ML%' and name not ilike '%HTML%')
or name ilike'%machine learning%'
or name ilike '%машинн%обучен%')
and (key_skills ilike '%sql%' 
or key_skills ilike '%postgres%')



'''

In [47]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

/tmp/ipykernel_4497/3310962512.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [48]:
# текст запроса
query_6_4 = f'''
select count(*) 
from VACANCIES
where
    (name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
    or (name ilike '%ML%' and name not ilike '%HTML%')
    or name ilike'%machine learning%'
    or name ilike '%машинн%обучен%')
    and key_skills ilike '%Python%'


'''

In [49]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

/tmp/ipykernel_4497/614304167.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [50]:
# текст запроса
query_6_5 = f'''
select  round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1),2) 
from VACANCIES
where (name ilike '%data%scientist%' 
or name ilike '%data%science%'
or name ilike '%исследователь%данных%'
or (name like '%ML%' and  name not ilike '%HTML%')
or name ilike '%machine%learning%'
or name ilike '%машинн%обучен%')
and key_skills is not null


'''

In [51]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

/tmp/ipykernel_4497/4089566367.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [105]:
# текст запроса
query_6_6 = f'''
select n.experience, round(avg(case
    when n.salary_from = 0 and n.salary_to <> 0 then n.salary_to
    when n.salary_from <> 0 and n.salary_to = 0 then n.salary_from
    when n.salary_from <> 0 and n.salary_to <> 0 then (n.salary_from + n.salary_to)/2
    end)) as "resoult"                   
from
(select experience,coalesce(salary_from,0) as "salary_from",coalesce(salary_to,0) as "salary_to"
from VACANCIES
where
    (name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not ilike '%HTML%')
    or name ilike'%machine learning%'
    or name ilike '%машинн%обучен%')
    and (salary_from <> 0 or salary_to <> 0)) as "n"
where n.experience = 'От 3 до 6 лет'
group by  experience
'''

In [106]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

/tmp/ipykernel_4497/2297777544.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,resoult
0,От 3 до 6 лет,243115.0


7. Написать запрос, где мы увидем лидеров размещения вакансий для DS.

In [112]:
query_6_7 =f'''
select e.name,count(*)
from VACANCIES as "v"
join AREAS as "a" on v.area_id=a.id
join EMPLOYERS as "e" on v.employer_id = e.id
where
    (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%'
    or (v.name ilike '%ML%' and v.name not ilike '%HTML%')
    or v.name ilike'%machine learning%'
    or v.name ilike '%машинн%обучен%') 
group by e.name   
order by 2 desc
limit 20


'''

In [113]:
df_6_7 = pd.read_sql_query(query_6_7, connection)
df_6_7

/tmp/ipykernel_4497/978001746.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count
0,СБЕР,39
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Контур,6


* Мы видим, что лидером по аналитике данных является СБЕР.

***

# выводы по предметному анализу
#### Что мы видим из запросов выше
* 1771 вакансия связана с данными из них 536 вакансий подходит для DS  и 51 вакансия расчитана на уровень Junior. Это на всю страну, включая страны СНГ. Уровень конкуренции высок.
* Из 536 вакансий, которые мы нашли подходящие для DS 229 содержат требования к знанию SQL и postgres
* А вот процент знания Python выше. тут уже 357 вакансий содержат требование к соискателю знания Python
* В среднем в требованиях указывается 6 навыков для DS
* Но и уровень зп тоже приятен. Даже на уровне новичка зп СРЕДНЯЯ ПО СТРАНЕ 74 тыс. Кстати, данный уровень соответствует минимальному среднему уровню по стране на основе данных из hh.ru. А вот уже человек с опытом получает выше чем средняя максимальная граница, что не может не радовать.
* И если мы ищем работу в данной сфере, то в запросе выше видны лидеры, предоставляющие вакансии на рынок. 

# Общий вывод по проекту

# подведем итог исследования, обобщите выводы
* Из всего вышеизложенного можно сделать несколько выводов:
  1. Для более детальной и полной картины необходимо, что бы работодатель размещал уровень зарплаты и сферы деятельности его компании.
  2. Крупный игрок на рынке не означает, что предлагает самую высокую зарплату, есть компании меньше и зарплату предлагает выше(тут мы не брали разные сферы деятельности).
  3. Тот факт, что работодатели уже более лояльно относятся к удаленному графику работы, нельзя не заметить. Количество вакансий растет с каждым годом.
  4. Если брать сферу IT, а конкретно должность DS, то большенсво компаний требуют знания python и SQL.
  5. Так же есть вакансии и для новичков, хоть их не так много, но учитывая, что сейчас много человеческих ресурсов задействовано в СВО, шансы устройства на работу увеличиваются.
  6. Сейчас сфера IT набирает популярность. Растут зарплаты и уровень знания в данной сфере. 
   